### Optuna Hyperparamter DeepSORT

In [1]:
!pip install optuna

     |████████████████████████████████| 308 kB 1.4 MB/s            
     |████████████████████████████████| 1.6 MB 2.0 MB/s            
     |████████████████████████████████| 210 kB 1.7 MB/s            
     |████████████████████████████████| 80 kB 559 kB/s             
     |████████████████████████████████| 150 kB 1.4 MB/s            
     |████████████████████████████████| 75 kB 231 kB/s            
     |████████████████████████████████| 49 kB 749 kB/s            
     |████████████████████████████████| 112 kB 1.9 MB/s            
     |████████████████████████████████| 149 kB 1.6 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11106 sha256=e01e86e3a0b2111109aa4c06de62a4b02d576e782eb6a6e11417c8833f7b721d
  Stored in directory: /home/ubuntu/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [1]:
import sys
import os
import argparse
import numpy as np
import optuna
import importlib

In [2]:
os.chdir(os.path.abspath('/home/ubuntu/ibrahim/master/models/research/object_detection/tracking/TrackEval/'))
import trackeval 

In [14]:
dataset_config = trackeval.datasets.MotChallenge2DBox.get_default_dataset_config()
dataset_config['GT_FOLDER'] = '/home/ubuntu/ibrahim/master/datasets/MOT_GT/'
dataset_config['TRACKERS_FOLDER'] = '/home/ubuntu/ibrahim/master/results/ssd_final_embeddings/'
dataset_config['CLASSES_TO_EVAL'] = ['vehicle']
dataset_config['DO_PREPROC'] = False

In [4]:
seq = 'MVI_39031'
tracker = 'data'
class_list = ['vehicle']
metrics_list = [trackeval.metrics.HOTA({'METRICS': ['HOTA'], 'THRESHOLD': 0.5})]
metric_names = ['HOTA']

In [5]:
os.chdir(os.path.abspath('/home/ubuntu/ibrahim/master/models/research/'))
from research.object_detection.tracking.deep_sort.deep_sort import nn_matching
from research.object_detection.tracking.deep_sort.deep_sort.detection import Detection
import research.object_detection.tracking.deep_sort.deep_sort.tracker
from research.object_detection.tracking.deep_sort.deep_sort.tracker import Tracker

In [15]:
def objective(trial):
    importlib.reload(research.object_detection.tracking.deep_sort.deep_sort.tracker)
    #matching_metric = trial.suggest_categorical("matching_metric", ["euclidean", "cosine"])
    matching_threshold = trial.suggest_float("matching_threshold", 0.1, 0.9)
    matching_budget = trial.suggest_int("matching_budget", 0, 10)
    tracker_max_iou_distance = trial.suggest_float("tracker_max_iou_distance", 0.3, 0.9)
    tracker_max_age = trial.suggest_int("tracker_max_age", 1, 50)
    tracker_n_init = trial.suggest_int("tracker_n_init", 1, 10)
    std_weight_position = trial.suggest_int("std_weight_position", 1, 200)
    std_weight_velocity = trial.suggest_int("std_weight_velocity", 1, 400)

    matching = nn_matching.NearestNeighborDistanceMetric("cosine", matching_threshold, matching_budget)
    deep_tracker = Tracker(matching, tracker_max_iou_distance, tracker_max_age, tracker_n_init, std_weight_position, std_weight_velocity)
    
    prediticon_file = open(os.path.join('/home/ubuntu/ibrahim/master/results/ssd_final_optuna/MOT17-train/data/data/', seq + ".txt"), "w")
    
    with open(os.path.join('/home/ubuntu/ibrahim/master/results/ssd_final_embeddings/MOT17-train/data/data/', seq + ".npy"), "rb") as prediticon_np:
        i=1
        while True:
            try:
                detections = np.load(prediticon_np, allow_pickle=True)
            except:
                break
            deep_tracker.predict()
            deep_tracker.update(detections)
            track_bbs_ids = []
            for track in deep_tracker.tracks:
                if not track.is_confirmed() or track.time_since_update > 1:
                    continue
                track_bbs_ids.append([*track.to_tlbr(), track.track_id])
            for track_pred in track_bbs_ids:
                bb_w = track_pred[2] - track_pred[0]
                bb_h = track_pred[3] - track_pred[1]
                prediticon_file.write(f'{i}, {int(track_pred[4])}, {track_pred[0]:.2f}, {track_pred[1]:.2f}, {bb_w:.2f}, {bb_h:.2f}, 1, -1, -1, -1 \n')
            i+=1
    prediticon_file.close()
    dataset = trackeval.datasets.MotChallenge2DBox(dataset_config)
    res = trackeval.eval.eval_sequence(seq, dataset, tracker, class_list, metrics_list, metric_names)
    return np.average(res['vehicle']['HOTA']['HOTA'])

In [16]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200) 

[I 2022-02-13 00:13:32,583] A new study created in memory with name: no-name-11d52b7e-c661-4d57-b0f2-74f6ce381695



MotChallenge2DBox Config:
GT_FOLDER            : /home/ubuntu/ibrahim/master/datasets/MOT_GT/
TRACKERS_FOLDER      : /home/ubuntu/ibrahim/master/results/ssd_final_embeddings/
OUTPUT_FOLDER        : None                          
TRACKERS_TO_EVAL     : None                          
CLASSES_TO_EVAL      : ['vehicle']                   
BENCHMARK            : MOT17                         
SPLIT_TO_EVAL        : train                         
INPUT_AS_ZIP         : False                         
PRINT_CONFIG         : True                          
DO_PREPROC           : False                         
TRACKER_SUB_FOLDER   : data                          
OUTPUT_SUB_FOLDER    :                               
TRACKER_DISPLAY_NAMES : None                          
SEQMAP_FOLDER        : None                          
SEQMAP_FILE          : None                          
SEQ_INFO             : None                          
GT_LOC_FORMAT        : {gt_folder}/{seq}/gt/gt.txt   
SKIP_SPLIT_FO

[I 2022-02-13 00:13:44,675] Trial 0 finished with value: 0.4381057919699379 and parameters: {'matching_threshold': 0.6700425654131404, 'matching_budget': 4, 'tracker_max_iou_distance': 0.6896927487725686, 'tracker_max_age': 43, 'tracker_n_init': 9, 'std_weight_position': 139, 'std_weight_velocity': 165}. Best is trial 0 with value: 0.4381057919699379.



MotChallenge2DBox Config:
GT_FOLDER            : /home/ubuntu/ibrahim/master/datasets/MOT_GT/
TRACKERS_FOLDER      : /home/ubuntu/ibrahim/master/results/ssd_final_embeddings/
OUTPUT_FOLDER        : None                          
TRACKERS_TO_EVAL     : None                          
CLASSES_TO_EVAL      : ['vehicle']                   
BENCHMARK            : MOT17                         
SPLIT_TO_EVAL        : train                         
INPUT_AS_ZIP         : False                         
PRINT_CONFIG         : True                          
DO_PREPROC           : False                         
TRACKER_SUB_FOLDER   : data                          
OUTPUT_SUB_FOLDER    :                               
TRACKER_DISPLAY_NAMES : None                          
SEQMAP_FOLDER        : None                          
SEQMAP_FILE          : None                          
SEQ_INFO             : None                          
GT_LOC_FORMAT        : {gt_folder}/{seq}/gt/gt.txt   
SKIP_SPLIT_FO

[I 2022-02-13 00:13:53,849] Trial 1 finished with value: 0.4381057919699379 and parameters: {'matching_threshold': 0.25868821238916506, 'matching_budget': 6, 'tracker_max_iou_distance': 0.5895740704302668, 'tracker_max_age': 50, 'tracker_n_init': 9, 'std_weight_position': 29, 'std_weight_velocity': 47}. Best is trial 0 with value: 0.4381057919699379.


...

[I 2022-02-13 01:02:23,470] Trial 197 finished with value: 0.4381057919699379 and parameters: {'matching_threshold': 0.5579628519025782, 'matching_budget': 5, 'tracker_max_iou_distance': 0.38017257661056714, 'tracker_max_age': 28, 'tracker_n_init': 8, 'std_weight_position': 106, 'std_weight_velocity': 102}. Best is trial 0 with value: 0.4381057919699379.



MotChallenge2DBox Config:
GT_FOLDER            : /home/ubuntu/ibrahim/master/datasets/MOT_GT/
TRACKERS_FOLDER      : /home/ubuntu/ibrahim/master/results/ssd_final_embeddings/
OUTPUT_FOLDER        : None                          
TRACKERS_TO_EVAL     : None                          
CLASSES_TO_EVAL      : ['vehicle']                   
BENCHMARK            : MOT17                         
SPLIT_TO_EVAL        : train                         
INPUT_AS_ZIP         : False                         
PRINT_CONFIG         : True                          
DO_PREPROC           : False                         
TRACKER_SUB_FOLDER   : data                          
OUTPUT_SUB_FOLDER    :                               
TRACKER_DISPLAY_NAMES : None                          
SEQMAP_FOLDER        : None                          
SEQMAP_FILE          : None                          
SEQ_INFO             : None                          
GT_LOC_FORMAT        : {gt_folder}/{seq}/gt/gt.txt   
SKIP_SPLIT_FO

[I 2022-02-13 01:02:38,499] Trial 198 finished with value: 0.4381057919699379 and parameters: {'matching_threshold': 0.6592074992270673, 'matching_budget': 5, 'tracker_max_iou_distance': 0.4204483093167555, 'tracker_max_age': 41, 'tracker_n_init': 6, 'std_weight_position': 131, 'std_weight_velocity': 211}. Best is trial 0 with value: 0.4381057919699379.



MotChallenge2DBox Config:
GT_FOLDER            : /home/ubuntu/ibrahim/master/datasets/MOT_GT/
TRACKERS_FOLDER      : /home/ubuntu/ibrahim/master/results/ssd_final_embeddings/
OUTPUT_FOLDER        : None                          
TRACKERS_TO_EVAL     : None                          
CLASSES_TO_EVAL      : ['vehicle']                   
BENCHMARK            : MOT17                         
SPLIT_TO_EVAL        : train                         
INPUT_AS_ZIP         : False                         
PRINT_CONFIG         : True                          
DO_PREPROC           : False                         
TRACKER_SUB_FOLDER   : data                          
OUTPUT_SUB_FOLDER    :                               
TRACKER_DISPLAY_NAMES : None                          
SEQMAP_FOLDER        : None                          
SEQMAP_FILE          : None                          
SEQ_INFO             : None                          
GT_LOC_FORMAT        : {gt_folder}/{seq}/gt/gt.txt   
SKIP_SPLIT_FO

[I 2022-02-13 01:02:50,111] Trial 199 finished with value: 0.4381057919699379 and parameters: {'matching_threshold': 0.7578439640944358, 'matching_budget': 6, 'tracker_max_iou_distance': 0.36619030527330915, 'tracker_max_age': 38, 'tracker_n_init': 8, 'std_weight_position': 84, 'std_weight_velocity': 93}. Best is trial 0 with value: 0.4381057919699379.


In [10]:
print(study.best_params)

{'matching_metric': 'cosine', 'matching_threshold': 0.7738349050722172, 'matching_budget': 4, 'tracker_max_iou_distance': 0.7459954575333289, 'tracker_max_age': 26, 'tracker_n_init': 5, 'std_weight_position': 11, 'std_weight_velocity': 241}


In [11]:
print(study.best_value)

0.4850312310837515
